CER022 - Create App Proxy certificate
=====================================

This notebook creates a certificate for the App Deploy Proxy endpoint.

Steps
-----

### Parameters

In [ ]:
import getpass

app_name = "app-proxy"
scaledset_name = "appproxy"
container_name = "app-service-proxy"
prefix_keyfile_name = "service-proxy"
common_name = "appproxy-svc"

country_name = "US"
state_or_province_name = "Illinois"
locality_name = "Chicago"
organization_name = "Contoso"
organizational_unit_name = "Finance"
email_address = f"{getpass.getuser().lower()}@contoso.com"

ssl_configuration_file = "service.openssl.cnf"

days = "398" # the number of days to certify the certificate for

test_cert_store_root = "/var/opt/secrets/test-certificates"

extendedKeyUsage = ""

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {} # Output in stderr known to be transient, therefore automatically retry
error_hints = {} # Output in stderr where a known SOP/TSG exists which will be HINTed for further help
install_hint = {} # The SOP to help install the executable if it cannot be found

def run(cmd, return_output=False, no_output=False, retry_count=0, base64_decode=False, return_as_json=False, regex_mask=None):
    """Run shell command, stream stdout, print stderr and optionally return output

    NOTES:

    1.  Commands that need this kind of ' quoting on Windows e.g.:

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='data-pool')].metadata.name}

        Need to actually pass in as '"':

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='"'data-pool'"')].metadata.name}

        The ' quote approach, although correct when pasting into Windows cmd, will hang at the line:
        
            `iter(p.stdout.readline, b'')`

        The shlex.split call does the right thing for each platform, just use the '"' pattern for a '
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    # When running `azdata sql query` on Windows, replace any \n in """ strings, with " ", otherwise we see:
    #
    #    ('HY090', '[HY090] [Microsoft][ODBC Driver Manager] Invalid string or buffer length (0) (SQLExecDirectW)')
    #
    if platform.system() == "Windows" and cmd.startswith("azdata sql query"):
        cmd = cmd.replace("\n", " ")

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # When running `kubectl`, if AZDATA_OPENSHIFT is set, use `oc`
    #
    if cmd.startswith("kubectl ") and "AZDATA_OPENSHIFT" in os.environ:
        cmd_actual[0] = cmd_actual[0].replace("kubectl", "oc")

    # To aid supportability, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    # Display an install HINT, so the user can click on a SOP to install the missing binary
    #
    if which_binary == None:
        print(f"The path used to search for '{cmd_actual[0]}' was:")
        print(sys.path)

        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    cmd_display = cmd
    if regex_mask is not None:
        regex = re.compile(regex_mask)
        cmd_display = re.sub(regex, '******', cmd)
        
    print(f"START: {cmd_display} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            try:
                line_decoded = line.decode()
            except UnicodeDecodeError:
                # NOTE: Sometimes we get characters back that cannot be decoded(), e.g.
                #
                #   \xa0
                #
                # For example see this in the response from `az group create`:
                #
                # ERROR: Get Token request returned http error: 400 and server 
                # response: {"error":"invalid_grant",# "error_description":"AADSTS700082: 
                # The refresh token has expired due to inactivity.\xa0The token was 
                # issued on 2018-10-25T23:35:11.9832872Z
                #
                # which generates the exception:
                #
                # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 179: invalid start byte
                #
                print("WARNING: Unable to decode stderr line, printing raw bytes:")
                print(line)
                line_decoded = ""
                pass
            else:

                # azdata emits a single empty line to stderr when doing an hdfs cp, don't
                # print this empty "ERR:" as it confuses.
                #
                if line_decoded == "":
                    continue
                
                print(f"STDERR: {line_decoded}", end='')

                if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                    exit_code_workaround = 1

                # inject HINTs to next TSG/SOP based on output in stderr
                #
                if user_provided_exe_name in error_hints:
                    for error_hint in error_hints[user_provided_exe_name]:
                        if line_decoded.find(error_hint[0]) != -1:
                            display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

                # Verify if a transient error, if so automatically retry (recursive)
                #
                if user_provided_exe_name in retry_hints:
                    for retry_hint in retry_hints[user_provided_exe_name]:
                        if line_decoded.find(retry_hint) != -1:
                            if retry_count < MAX_RETRIES:
                                print(f"RETRY: {retry_count} (due to: {retry_hint})")
                                retry_count = retry_count + 1
                                output = run(cmd, return_output=return_output, retry_count=retry_count)

                                if return_output:
                                    if base64_decode:
                                        import base64
                                        return base64.b64decode(output).decode('utf-8')
                                    else:
                                        return output

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        if base64_decode:
            import base64
            return base64.b64decode(output).decode('utf-8')
        else:
            return output



# Hints for tool retry (on transient fault), known errors and install guide
#
retry_hints = {'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',  ], 'python': [ ], 'azdata': ['Endpoint sql-server-master does not exist', 'Endpoint livy does not exist', 'Failed to get state for cluster', 'Endpoint webhdfs does not exist', 'Adaptive Server is unavailable or does not exist', 'Error: Address already in use', 'Login timeout expired (0) (SQLDriverConnect)', 'SSPI Provider: No Kerberos credentials available',  ], }
error_hints = {'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb'],  ], 'python': [['Library not loaded: /usr/local/opt/unixodbc', 'SOP012 - Install unixodbc for Mac', '../install/sop012-brew-install-odbc-for-sql-server.ipynb'], ['WARNING: You are using pip version', 'SOP040 - Upgrade pip in ADS Python sandbox', '../install/sop040-upgrade-pip.ipynb'],  ], 'azdata': [['Please run \'azdata login\' to first authenticate', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['The token is expired', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Reason: Unauthorized', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Max retries exceeded with url: /api/v1/bdc/endpoints', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Look at the controller logs for more details', 'TSG027 - Observe cluster deployment', '../diagnose/tsg027-observe-bdc-create.ipynb'], ['provided port is already allocated', 'TSG062 - Get tail of all previous container logs for pods in BDC namespace', '../log-files/tsg062-tail-bdc-previous-container-logs.ipynb'], ['Create cluster failed since the existing namespace', 'SOP061 - Delete a big data cluster', '../install/sop061-delete-bdc.ipynb'], ['Failed to complete kube config setup', 'TSG067 - Failed to complete kube config setup', '../repair/tsg067-failed-to-complete-kube-config-setup.ipynb'], ['Data source name not found and no default driver specified', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Can\'t open lib \'ODBC Driver 17 for SQL Server', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Control plane upgrade failed. Failed to upgrade controller.', 'TSG108 - View the controller upgrade config map', '../diagnose/tsg108-controller-failed-to-upgrade.ipynb'], ['NameError: name \'azdata_login_secret_name\' is not defined', 'SOP013 - Create secret for azdata login (inside cluster)', '../common/sop013-create-secret-for-azdata-login.ipynb'], ['ERROR: No credentials were supplied, or the credentials were unavailable or inaccessible.', 'TSG124 - \'No credentials were supplied\' error from azdata login', '../repair/tsg124-no-credentials-were-supplied.ipynb'], ['Please accept the license terms to use this product through', 'TSG126 - azdata fails with \'accept the license terms to use this product\'', '../repair/tsg126-accept-license-terms.ipynb'],  ], }
install_hint = {'kubectl': [ 'SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb' ],  'azdata': [ 'SOP063 - Install azdata CLI (using package manager)', '../install/sop063-packman-install-azdata.ipynb' ],  }


print('Common functions defined successfully.')

### Get the Kubernetes namespace for the big data cluster

Get the namespace of the Big Data Cluster use the kubectl command line
interface .

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = run(f'kubectl get namespace --selector=MSSQL_CLUSTER -o jsonpath={{.items[0].metadata.name}}', return_output=True)
    except:
        from IPython.display import Markdown
        print(f"ERROR: Unable to find a Kubernetes namespace with label 'MSSQL_CLUSTER'.  SQL Server Big Data Cluster Kubernetes namespaces contain the label 'MSSQL_CLUSTER'.")
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print(f'The SQL Server Big Data Cluster Kubernetes namespace is: {namespace}')

### Create a temporary directory to stage files

In [ ]:
# Create a temporary directory to hold configuration files

import tempfile

temp_dir = tempfile.mkdtemp()

print(f"Temporary directory created: {temp_dir}")

### Helper function to save configuration files to disk

In [ ]:
# Define helper function 'save_file' to save configuration files to the temporary directory created above
import os
import io

def save_file(filename, contents):
    with io.open(os.path.join(temp_dir, filename), "w", encoding='utf8', newline='\n') as text_file:
      text_file.write(contents)

      print("File saved: " + os.path.join(temp_dir, filename))

print("Function `save_file` defined successfully.")

### Get endpoint hostname

In [ ]:
import json
import urllib

endpoint_name = "sql-server-master" if app_name == "master" else app_name 

endpoint = run(f'azdata bdc endpoint list --endpoint="{endpoint_name}"', return_output=True)
endpoint = json.loads(endpoint)
endpoint = endpoint['endpoint']

print(f"endpoint: {endpoint}")

if app_name == "master":
    hostname = endpoint.split(",")[0]
else:
    hostname = urllib.parse.urlparse(endpoint).hostname

print(f"hostname: {hostname}")

### Get name of the ‘Running’ `controller` `pod`

In [ ]:
# Place the name  of the 'Running' controller pod in variable `controller`

controller = run(f'kubectl get pod --selector=app=controller -n {namespace} -o jsonpath={{.items[0].metadata.name}} --field-selector=status.phase=Running', return_output=True)

print(f"Controller pod name: {controller}")

### Get the name of the pod.

In [ ]:
# Get the name of the pod

if (app_name == "mgmtproxy"):
  pod = run(f'kubectl get pod --selector=app=mgmtproxy -n {namespace} -o jsonpath={{.items[0].metadata.name}}', return_output=True)
elif (app_name == "gateway"):
  pod = run(f'kubectl get pod --selector=app=gateway -n {namespace} -o jsonpath={{.items[0].metadata.name}}', return_output=True)
elif (app_name == "app-proxy"): 
  pod = run(f'kubectl get pod --selector=app=appproxy -n {namespace} -o jsonpath={{.items[0].metadata.name}}', return_output=True)
elif (app_name == "controller"):
  pod = run(f'kubectl get pod --selector=app=mgmtproxy -n {namespace} -o jsonpath={{.items[0].metadata.name}}', return_output=True)
elif (app_name == "master"):
  # Use explicitly provided pod name for master
  #  
  pod = master_pod_name
else:
  raise SystemExit(f'Invalid app name')

print(f"Pod name: {pod}")

### Create the DNS alt\_names for data plane in secure clusters

Get the cluster configuration from the Big Data Cluster using
`azdata bdc config`, and pull the Active Directory DNS names out of it,
and place them into the certificate configuration file as DNS alt\_names

In [ ]:
import json

alt_names = ""
bdc_fqdn = ""

hdfs_vault_svc = "hdfsvault-svc"
mssql_vault_svc = "mssqlvault-svc"
bdc_config = run("azdata bdc config show", return_output=True)
bdc_config = json.loads(bdc_config)

dns_counter = 3 # DNS.1 and DNS.2 are already in the certificate template.

if app_name == "gateway" or app_name == "master":
  alt_names += f'DNS.{str(dns_counter)} = {pod}.{common_name}\n'
  dns_counter = dns_counter + 1
  alt_names += f'DNS.{str(dns_counter)} = {pod}.{common_name}.{namespace}.svc.cluster.local\n'
  dns_counter = dns_counter + 1

if "security" in bdc_config["spec"] and "activeDirectory" in bdc_config["spec"]["security"]:
    domain_dns_name = bdc_config["spec"]["security"]["activeDirectory"]["domainDnsName"]
    sub_domain_name = bdc_config["spec"]["security"]["activeDirectory"]["subdomain"]

    if sub_domain_name:
        bdc_fqdn = f"{sub_domain_name}.{domain_dns_name}"
    else:
        bdc_fqdn = f"{namespace}.{domain_dns_name}"

    alt_names += f"DNS.{str(dns_counter)} = {common_name}.{bdc_fqdn}\n"
    dns_counter = dns_counter + 1

    if app_name == "gateway" or app_name == "master":
      alt_names += f'DNS.{str(dns_counter)} = {pod}.{bdc_fqdn}\n'
      dns_counter = dns_counter + 1

if app_name in bdc_config["spec"]["resources"]:
    app_name_endpoints = bdc_config["spec"]["resources"][app_name]["spec"]["endpoints"]
    for endpoint in app_name_endpoints:
        if "dnsName" in endpoint:
            alt_names += f'DNS.{str(dns_counter)} = {endpoint["dnsName"]}\n'
            dns_counter = dns_counter + 1

# Special case for the controller certificate
#
if app_name == "controller":
    alt_names += f"DNS.{str(dns_counter)} = localhost\n"
    dns_counter = dns_counter + 1

    # Add hdfsvault-svc host for key management calls.
    #
    alt_names += f"DNS.{str(dns_counter)} = {hdfs_vault_svc}\n"
    dns_counter = dns_counter + 1
  
    # Add mssqlvault-svc host for key management calls.
    #
    alt_names += f"DNS.{str(dns_counter)} = {mssql_vault_svc}\n"
    dns_counter = dns_counter + 1

    # Add hdfsvault-svc FQDN for key management calls.
    #
    if bdc_fqdn:
        alt_names += f"DNS.{str(dns_counter)} = {hdfs_vault_svc}.{bdc_fqdn}\n"
        dns_counter = dns_counter + 1
      
    # Add mssqlvault-svc FQDN for key management calls.
    #
    if bdc_fqdn:
        alt_names += f"DNS.{str(dns_counter)} = {mssql_vault_svc}.{bdc_fqdn}\n"
        dns_counter = dns_counter + 1

print("DNS alt_names (data plane):")
print(alt_names)

### Create the DNS alt\_names for control plane in secure clusters

Get the cluster configuration from the Big Data Cluster using
`azdata bdc endpoint list`, and pull the Active Directory DNS names out
of it for the control plane expternal endpoints (Controller and
Management Proxy), and place them into the certificate configuration
file as DNS alt\_names

In [ ]:
import json
from urllib.parse import urlparse

if app_name == "controller" or app_name == "mgmtproxy":
    bdc_endpoint_list = run("azdata bdc endpoint list", return_output=True)
    bdc_endpoint_list = json.loads(bdc_endpoint_list)

    # Parse the DNS host name from:
    #
    #    "endpoint": "https://monitor.aris.local:30777"
    # 
    for endpoint in bdc_endpoint_list:
        if endpoint["name"] == app_name:
            url = urlparse(endpoint["endpoint"])
            alt_names += f"DNS.{str(dns_counter)} = {url.hostname}\n"
            dns_counter = dns_counter + 1

print("DNS alt_names (control plane):")
print(alt_names)

### Create alt\_names

If the Kuberenetes service is of “NodePort” type, then the IP address
needed to validate the cluster certificate could be for any node in the
Kubernetes cluster, so here all node IP addresses in the Big Data
Cluster are added as alt\_names. Otherwise (if not NodePort, and
therefore LoadBalancer), add just the hostname as returned from
`azdata bdc endpoint list` above.

In [ ]:
service_type = run(f"kubectl get svc {common_name}-external -n {namespace} -o jsonpath={{.spec.type}}", return_output=True)

print(f"Service type for '{common_name}-external' is: '{service_type}'")
print("")

if service_type == "NodePort":
    nodes_ip_address = run("kubectl ""get nodes -o jsonpath={.items[*].status.addresses[0].address}""", return_output=True)
    nodes_ip_address = nodes_ip_address.split(' ')

    counter = 1
    for ip in nodes_ip_address:
        alt_names += f"IP.{counter} = {ip}\n"
        counter = counter + 1
else:
    alt_names += f"IP.1 = {hostname}\n"

print("All (DNS and IP) alt_names:")
print(alt_names)

### Generate Certificate Configuration file

NOTE: There is a special case for the `controller` certificate, that
needs to be generated in PKCS\#1 format.

In [ ]:
certificate = f"""
[ req ]
# Options for the `req` tool (`man req`).
default_bits        = 2048
default_keyfile     = {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-privatekey{".pkcs8" if app_name == "controller" else ""}.pem
distinguished_name  = req_distinguished_name
string_mask         = utf8only

# SHA-1 is deprecated, so use SHA-2 instead.
default_md          = sha256
req_extensions      = v3_req

[ req_distinguished_name ]
countryName         = Country Name (2 letter code)
countryName_default = {country_name}

stateOrProvinceName         = State or Province Name (full name)
stateOrProvinceName_default = {state_or_province_name}

localityName                = Locality Name (eg, city)
localityName_default        = {locality_name}

organizationName            = Organization Name (eg, company)
organizationName_default    = {organization_name}

organizationalUnitName         = Organizational Unit (eg, division)
organizationalUnitName_default = {organizational_unit_name}

commonName         = Common Name (e.g. server FQDN or YOUR name)
commonName_default = {common_name}

emailAddress         = Email Address
emailAddress_default = {email_address}


[ v3_req ]
subjectAltName       = @alt_names
subjectKeyIdentifier = hash
basicConstraints     = CA:FALSE
keyUsage             = digitalSignature, keyEncipherment
{extendedKeyUsage}

[ alt_names ]
DNS.1 = {common_name}
DNS.2 = {common_name}.{namespace}.svc.cluster.local  # Use the namespace applicable for your cluster
{alt_names}
"""

print(certificate)

save_file(ssl_configuration_file, certificate)

### Copy certificate configuration to `controller` `pod`

In [ ]:
import os

cwd = os.getcwd()
os.chdir(temp_dir) # Use chdir to workaround kubectl bug on Windows, which incorrectly processes 'c:\' on kubectl cp cmd line 

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "mkdir -p {test_cert_store_root}/{app_name}"')

run(f'kubectl cp {ssl_configuration_file} {controller}:{test_cert_store_root}/{app_name}/{ssl_configuration_file} -c controller -n {namespace}')

os.chdir(cwd)

### Generate certificate

Use openssl req to generate a certificate in PKCS\#10 format. See:

-   https://www.openssl.org/docs/man1.0.2/man1/req.html

In [ ]:
cmd = f"openssl req -config {test_cert_store_root}/{app_name}/service.openssl.cnf -newkey rsa:2048 -sha256 -nodes -days {days} -out {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-signingrequest.csr -outform PEM -subj '/C={country_name}/ST={state_or_province_name}/L={locality_name}/O={organization_name}/OU={organizational_unit_name}/CN={common_name}'"

run(f'kubectl exec {controller} -n {namespace} -c controller -- bash -c "{cmd}"')

### Clean up temporary directory for staging configuration files

In [ ]:
# Delete the temporary directory used to hold configuration files

import shutil

shutil.rmtree(temp_dir)

print(f'Temporary directory deleted: {temp_dir}')

In [ ]:
print("Notebook execution is complete.")

Related
-------

- [CER030 - Sign Management Proxy certificate with generated CA](../cert-management/cer030-sign-service-proxy-generated-cert.ipynb)
- [CER032 - Sign App-Proxy certificate with generated CA](../cert-management/cer032-sign-app-proxy-generated-cert.ipynb)
- [CER042 - Install signed App-Proxy certificate](../cert-management/cer042-install-app-proxy-cert.ipynb)
